In [1]:
import json
import os
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook

In [8]:
files = os.listdir('tvk')
first = True

for file in tqdm_notebook(files):
    with open(os.path.join('tvk',file),'r', encoding="utf8") as cur_file:
        parsed_file = json.load(cur_file)
        if first:
            tvos_all = parsed_file
            first = False
        else:
            tvos_all['features'] = tvos_all['features'] + parsed_file['features']
            
len(tvos_all['features'])

225

In [9]:
dtype_dic = {'district_number': str}
df_results = pd.read_csv('00_data/elections_round2_all_v2.csv',dtype=dtype_dic)
df_results.shape

(29608, 68)

In [10]:
df_results.columns

Index(['district_number', 'district_gen_numeric', 'district_general',
       'district_location', '1.К-сть одержаних бюлетенів',
       '2.К-сть виборців, внесених до списку_x',
       '3.К-сть виборців, внесених до витягу для голос. за місцем перебув.',
       '4.К-сть невикор. бюлетенів',
       '5.К-сть виборців, які отримали бюлетені у приміщ.',
       '6.К-сть виборців, які отримали бюлетені за місцем перебув.',
       '7.Загальна к-сть виборців, які отримали бюлетені',
       '8.К-сть бюлетенів, що не підлягають врахуванню',
       '9.К-сть виборців, які взяли участь у голосуванні',
       '10.К-сть бюлетенів, визнаних недійсними', 'ЗеленськийВолодимир_round2',
       'ПорошенкоПетро_round2', 'Дата та час протоколу',
       '2.К-сть виборців, внесених до списку', 'reg_voters_change',
       'reg_voters_change_abs', 'participation_round2', 'participation_round1',
       'ЗеленськийВолодимир', 'ПорошенкоПетро', 'БойкоЮрій', 'ВілкулОлександр',
       'ГриценкоАнатолій', 'ЛяшкоОлег',

In [11]:
df_results['participation_absolute_round1'] = df_results['2.К-сть виборців, внесених до списку']*df_results['participation_round1']
df_results['participation_absolute_round2'] = df_results['2.К-сть виборців, внесених до списку']*df_results['participation_round2']
df_results_tvo = df_results.groupby('district_gen_numeric')[['2.К-сть виборців, внесених до списку',
                                                             'participation_absolute_round1',
                                                             'participation_absolute_round2',
                                                             'ЗеленськийВолодимир',
                                                             'ПорошенкоПетро',
                                                             'ЗеленськийВолодимир_round2',
                                                             'ПорошенкоПетро_round2']].sum()
df_results_tvo['participation_diff'] = (df_results_tvo['participation_absolute_round2']-df_results_tvo['participation_absolute_round1'])/df_results_tvo['2.К-сть виборців, внесених до списку']
df_results_tvo['winner'] = (df_results_tvo['ЗеленськийВолодимир_round2'] - df_results_tvo['ПорошенкоПетро_round2']).map(lambda x: 'zel' if x>0 else 'por')
df_results_tvo['zel_growth'] = (df_results_tvo['ЗеленськийВолодимир_round2'] - df_results_tvo['ЗеленськийВолодимир'])/df_results_tvo['2.К-сть виборців, внесених до списку']
df_results_tvo['por_growth'] = (df_results_tvo['ПорошенкоПетро_round2'] - df_results_tvo['ПорошенкоПетро'])/df_results_tvo['2.К-сть виборців, внесених до списку']

print(df_results_tvo.shape)
df_results_tvo.head()

(200, 11)


,"2.К-сть виборців, внесених до списку",participation_absolute_round1,participation_absolute_round2,ЗеленськийВолодимир,ПорошенкоПетро,ЗеленськийВолодимир_round2,ПорошенкоПетро_round2,participation_diff,winner,zel_growth,por_growth
district_gen_numeric,,,,,,,,,,,
11,163874,111260.0,105176.0,25515,31879,60318.711895,44857.288105,-0.037126,zel,0.212381,0.079197
12,174795,113698.0,107943.0,28056,31292,63104.810376,44838.189624,-0.032924,zel,0.200514,0.077498
13,145532,95070.0,90401.0,22149,20453,58151.268775,32249.731225,-0.032082,zel,0.247384,0.081059
14,159494,102258.0,98082.0,24356,23232,63434.103448,34647.896552,-0.026183,zel,0.245013,0.071576
15,148528,93001.0,88213.0,21896,18318,60228.000000,27985.000000,-0.032236,zel,0.258079,0.065085


In [12]:
for feature in tqdm_notebook(tvos_all['features']):
    if int(feature['properties']['id']) in df_results_tvo.index:
        feature['properties']['participation_diff'] = df_results_tvo.loc[int(feature['properties']['id'])]['participation_diff']
        feature['properties']['zel_growth'] = df_results_tvo.loc[int(feature['properties']['id'])]['zel_growth']
        feature['properties']['por_growth'] = df_results_tvo.loc[int(feature['properties']['id'])]['por_growth']
        feature['properties']['winner'] = df_results_tvo.loc[int(feature['properties']['id'])]['winner']
    else:
        feature['properties']['participation_diff'] = 9999
        feature['properties']['zel_growth'] = 9999
        feature['properties']['por_growth'] = 9999
        feature['properties']['winner'] = 'no_vote'

len(tvos_all['features'])

225

In [13]:
with open('01_map/tvos_all_v6.json','w', encoding="utf8") as output:
    json.dump(tvos_all,output,ensure_ascii=False)

In [84]:
with open('newElectionData.json','r',encoding="utf8") as file:
    districts_all = json.load(file)

In [118]:
district = districts_all['objects']['-']['geometries'][1300]

for district in tqdm_notebook(districts_all['objects']['-']['geometries']):
    if district['properties']['d'] in list(df_results['district_number']):
        district['properties']['participation_diff'] = df_results.loc[df_results['district_number']==district['properties']['d']].reset_index()['participation_delta'][0]
    else:
        district['properties']['participation_diff'] = 9999

In [126]:
with open('newElectionData_v2.json','w',encoding="utf8") as file:
    json.dump(districts_all,file,ensure_ascii=False)